# week 11  과제
## 컴퓨터 과학과 201710957 이유진

## 연습문제: 은행 송금



은행은 자신이 계정을 개설하여, 입출금을 할 수 있다.

컨트랙을 프로그램하여 아래 기능을 실행하도록 하세요.

주피터 노트북으로 프로그램하세요.



은행 컨트랙의 함수는 다음과 같이 만드세요.

* 입금액을 정해서 입금

	function deposit(uint amount) public payable

* 전액 인출 함수

	function widthdrawAll() public

* 컨트랙 잔고확인 함수 (this를 이용한 잔고, 상태변수 잔고 (this 잔고가 맞는지 확인하는 용도)

	function getBalance() public view returns(uint, uint)

* 다른 계정으로 계좌이체 함수

	function forwardTo(address payable _receiver) public payable



아래 기능을 실행해 보세요.

- 입금 11111 wei, 222 wei를 하고 잔고 11333 wei 출력

- 자신의 2번째 계정으로 계좌이체 333 wei하고 컨트랙 잔고 11000 wei 출력, 자신의 2번째 계정 잔고 증가분 (+333 wei) 출력

- 전액 인출하고 11000 wei, 자신의 잔고 증가분 출력 (+11000 wei)

coinbase와 두번째 계정 확인

In [62]:
!geth --exec "eth.coinbase" attach http://localhost:8445

"0x478e5deacb2c8b60452f7d9822e50643bb78c1d3"


In [61]:
!geth --exec "eth.accounts[1]" attach http://localhost:8445

"0x50ecededd9c68a523fbde5b8b41777b6704ec717"


Coinibase, 두번째 계정 잔고 출력 코드 작성

In [243]:
%%writefile C:\Users\sofod\Code\201710957\src\mybank_checkCB.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));


var abi = [{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"widthdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var addr = "0x5F1acfbc60fD05625bF47fE8e944A1cBA6698282";
var mybank = new web3.eth.Contract(abi,addr);


var coinbase = "0x478e5deacb2c8b60452f7d9822e50643bb78c1d3";
web3.eth.getBalance(coinbase).then(console.log);


Overwriting C:\Users\sofod\Code\201710957\src\mybank_checkCB.js


In [250]:
%%writefile C:\Users\sofod\Code\201710957\src\mybank_checkAccount2.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));


var abi = [{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"widthdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var addr = "0x5F1acfbc60fD05625bF47fE8e944A1cBA6698282";
var mybank = new web3.eth.Contract(abi,addr);


web3.eth.getBalance("0x50ecededd9c68a523fbde5b8b41777b6704ec717").then(console.log);


Overwriting C:\Users\sofod\Code\201710957\src\mybank_checkAccount2.js


## 1) Solidity 코딩

In [160]:
%%writefile C:\Users\sofod\Code\201710957\src\mybank.sol
pragma solidity ^0.5.0;

contract MyBank{
    address payable owner;
    uint balance;
    constructor() public {
        owner = msg.sender;
        balance = address(this).balance;
    }
    function deposit(uint amount) public payable {
        require( msg.value == amount );
        balance += amount;
    } 
    
    function widthdrawAll() public {
        uint amount = balance;
        balance -= amount;
        owner.transfer(amount);
    }
    function getBalance() public view returns(uint, uint){
        return ( address(this).balance,  balance);
    }
    function forwardTo(address payable _receiver) public payable{
         uint amount = 333 wei;
        balance -= amount;
        _receiver.transfer(amount);
    }

   
}

Overwriting C:\Users\sofod\Code\201710957\src\mybank.sol


## 2) Solc를 이용하여 컴파일

In [161]:
!solc --version

solc, the solidity compiler commandline interface
Version: 0.5.0+commit.1d4f565a.Windows.msvc


In [162]:
!solc --abi --bin --gas C:\Users\sofod\Code\201710957\src\mybank.sol


======= C:/Users/sofod/Code/201710957/src/mybank.sol:MyBank =======
Gas estimation:
construction:
   40856 + 126600 = 167456
external:
   deposit(uint256):	20520
   forwardTo(address):	infinite
   getBalance():	841
   widthdrawAll():	infinite
Binary: 
608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055503073ffffffffffffffffffffffffffffffffffffffff16316001819055506102798061007e6000396000f3fe608060405260043610610062576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff16806312065fe01461006757806327d8ad88146100995780633c459375146100dd578063b6b55f25146100f4575b600080fd5b34801561007357600080fd5b5061007c610122565b604051808381526020018281526020019250505060405180910390f35b6100db600480360360208110156100af57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190505050610148565b005b3480156100e957600080fd5b506100f26101aa565b005b61012

## 3) node 를 이용한 배포

In [205]:
!geth --exec "eth.coinbase" attach http://localhost:8445

"0x478e5deacb2c8b60452f7d9822e50643bb78c1d3"


In [206]:
%%writefile C:\Users\sofod\Code\201710957\src\mybank_deploy.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var abi = [{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"widthdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var bankContract = new web3.eth.Contract(abi);
var data_ = '0x'+'608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055503073ffffffffffffffffffffffffffffffffffffffff16316001819055506102798061007e6000396000f3fe608060405260043610610062576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff16806312065fe01461006757806327d8ad88146100995780633c459375146100dd578063b6b55f25146100f4575b600080fd5b34801561007357600080fd5b5061007c610122565b604051808381526020018281526020019250505060405180910390f35b6100db600480360360208110156100af57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190505050610148565b005b3480156100e957600080fd5b506100f26101aa565b005b6101206004803603602081101561010a57600080fd5b810190808035906020019092919050505061022c565b005b6000803073ffffffffffffffffffffffffffffffffffffffff1631600154915091509091565b600061014d9050806001600082825403925050819055508173ffffffffffffffffffffffffffffffffffffffff166108fc829081150290604051600060405180830381858888f193505050501580156101a5573d6000803e3d6000fd5b505050565b60006001549050806001600082825403925050819055506000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff166108fc829081150290604051600060405180830381858888f19350505050158015610228573d6000803e3d6000fd5b5050565b803414151561023a57600080fd5b806001600082825401925050819055505056fea165627a7a7230582096ac8f48632254d86c05e52a169cfd32c1275dfbe0ce9e905eee7f7259c1e9080029';

bankContract
    .deploy({
            data: data_
    })
    .send({
     from: "0x478e5deacb2c8b60452f7d9822e50643bb78c1d3",
     gas: '1000000'
    }, function (error, transactionHash){ 
            console.log("error :" + error);
            console.log("hash : " + transactionHash); 
    })
    .then(function(newContractInstance){
        console.log("address : " + newContractInstance.options.address)
    });

Overwriting C:\Users\sofod\Code\201710957\src\mybank_deploy.js


In [262]:
!node C:\Users\sofod\Code\201710957\src\mybank_deploy.js

error :null
hash : 0xb4b2badd5dc946793e8ddb302cf57ca0408999320536cb0726a4aa3a667d36cb
address : 0x646507260fa423409C4493E0758fc64FaA196918


## 4) API 사용 contractAddress 호출하여 사용

###  입금 11111wei, 222 wei를 하고 잔고 111333wei 출력

코드작성

In [263]:
%%writefile C:\Users\sofod\Code\201710957\src\mybankUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));

var abi = [{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"widthdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var addr = "0x646507260fa423409C4493E0758fc64FaA196918";
var mybank = new web3.eth.Contract(abi,addr);

var coinbase = "0x478e5deacb2c8b60452f7d9822e50643bb78c1d3";
mybank.methods.deposit(11111).send({from:coinbase, gas:80000,value :11111});
mybank.methods.deposit(222).send({from:coinbase, gas:80000,value :222});



Overwriting C:\Users\sofod\Code\201710957\src\mybankUse.js


In [264]:
%%writefile C:\Users\sofod\Code\201710957\src\mybankUse_checkBalacne.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));

var abi = [{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"widthdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var addr = "0x646507260fa423409C4493E0758fc64FaA196918";
var mybank = new web3.eth.Contract(abi,addr);

mybank.methods.getBalance().call().then(console.log);



Overwriting C:\Users\sofod\Code\201710957\src\mybankUse_checkBalacne.js


1. 입금(11111 wei, 222 wei) 

In [265]:
!node C:\Users\sofod\Code\201710957\src\mybankUse.js

2. 잔고출력

In [266]:
!node C:\Users\sofod\Code\201710957\src\mybankUse_checkBalacne.js

Result { '0': '11333', '1': '11333' }


### 자신의 2번째 계정으로 계좌이체 333 wei 하고 컨트랙잔고 111000wei 출력, 자신의 2번째 잔고 증가분(+333 wei) 출력



코드작성

In [267]:
%%writefile C:\Users\sofod\Code\201710957\src\mybankUse_forwardTo.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));


var abi = [{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"widthdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var addr = "0x646507260fa423409C4493E0758fc64FaA196918";
var mybank = new web3.eth.Contract(abi,addr);


var coinbase = "0x478e5deacb2c8b60452f7d9822e50643bb78c1d3";
var receiver = "0x50ecededd9c68a523fbde5b8b41777b6704ec717";

mybank.methods.forwardTo(receiver).send({from:coinbase});


Overwriting C:\Users\sofod\Code\201710957\src\mybankUse_forwardTo.js


계좌이체 전 두번째 계정의 잔고, 컨트랙잔고 출력

In [268]:
!node C:\Users\sofod\Code\201710957\src\mybank_checkAccount2.js

130000000000000165994


In [269]:
!node C:\Users\sofod\Code\201710957\src\mybankUse_checkBalacne.js

Result { '0': '11333', '1': '11333' }


계좌이체

In [271]:
!node C:\Users\sofod\Code\201710957\src\mybankUse_forwardTo.js

계좌이체후 잔고출력

In [272]:
!node C:\Users\sofod\Code\201710957\src\mybankUse_checkBalacne.js

Result { '0': '11000', '1': '11000' }


In [273]:
!node C:\Users\sofod\Code\201710957\src\mybank_checkAccount2.js

130000000000000166327


 130000000000000166327 - 130000000000000165994 = 333 으로 
 333 wei 증가하였다


###  전액 인출하고 11000wei, 자신의 잔고 증가분 출력 (+11000 wei)

코드작성

In [275]:
%%writefile C:\Users\sofod\Code\201710957\src\mybankUse_withdrawAll.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));



var abi = [{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"widthdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var addr = "0x646507260fa423409C4493E0758fc64FaA196918";
var mybank = new web3.eth.Contract(abi,addr);

var coinbase = "0x478e5deacb2c8b60452f7d9822e50643bb78c1d3";
mybank.methods.widthdrawAll().send({from:coinbase});


Overwriting C:\Users\sofod\Code\201710957\src\mybankUse_withdrawAll.js


전액 인출 전 잔고

In [276]:
!node C:\Users\sofod\Code\201710957\src\mybank_checkCB

1673656249909999685342


전액 인출

In [277]:
!node C:\Users\sofod\Code\201710957\src\mybankUse_withdrawAll.js

전액 인출 후 코인베이스 잔고와 컨트랙 잔고

In [278]:
!node C:\Users\sofod\Code\201710957\src\mybank_checkCB

1678656249909999696342


In [279]:
!node C:\Users\sofod\Code\201710957\src\mybankUse_checkBalacne.js

Result { '0': '0', '1': '0' }


1678656249909999696342-1673656249909999685342 = 5000000000000011000 으로, 

pending을 해제하기 위해서 miner.start(); admin.sleepBlocks(1); miner.stop() 를 시행하였기 때문에

11000 wei보다 더 많은 금액이 증가하였다.

하지만 1block을 마이낭하였기에 5000000000000000000을 무시하면 11000 wei가 송금됨을 알수있다.